In [1]:
!pip install tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 33kB/s 
     |████████████████████████████████| 3.1MB 37.5MB/s 
     |████████████████████████████████| 501kB 37.1MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

# Get the data
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

2.0.0-beta1


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/0.1.0. Subsequent calls will reuse this data.


In [0]:
tokenizer = info.features['text'].encoder

BUFFER_SIZE = 10000
BATCH_SIZE = 64
# tf.compat.v1.data.get_output_shapes(train_dataset)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))
test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset))

In [3]:
model = tf.keras.Sequential([
    # 64 is the batch size: number of outputs that I desire from that layer
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    # The bi directinal acts as a bridge to replace the Flatten and Avg Pooling 1D
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    # Note that we can stack bi directional LSTM BUT DO NOT FOR GET to specify
    # return_sequences = True in the PREVIOUS layer to ensure the output match 
    # the input shape
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
_________________________________________________________________


Note that the bi directional makes the output double the desired size (64) as 128

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

NUM_EPOCHS = 10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


391/391 [==============================] - 2809s 7s/step - loss: 0.6397 - accuracy: 0.6397 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
  9/391 [..............................] - ETA: 27:01 - loss: 0.7991 - accuracy: 0.4896

### WITH LSTM

In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [0]:
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

### WITHOUT LSTM

In [0]:
model = tf.keras.Sequential([
    # 64 is the batch size: number of outputs that I desire from that layer
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    # The bi directinal acts as a bridge to replace the Flatten and Avg Pooling 1D
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    # Note that we can stack bi directional LSTM BUT DO NOT FOR GET to specify
    # return_sequences = True in the PREVIOUS layer to ensure the output match 
    # the input shape
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [0]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

NUM_EPOCHS = 10

history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

- WITHOUT LSTM: For a trainee using the sarcasm data-set with these, when just using the pooling and flattening, I quickly got close to 85 percent accuracy and then it flattened out there. The validation set was a little less accurate, but the curves we're quite in sync.

  The loss values from my non-LSTM one got to healthy state quite quickly and then flattened out.

- WITH LSTM: On the other hand, when using LSTM, I reached 85 percent accuracy really quickly and continued climbing towards about 97.5 percent accuracy within 50 epochs. The validation set dropped slowly, but it was still close to the same value as the non- LSTM version. Still the drop indicates that there's some over fitting going on here. (While the accuracy of the prediction increased, the confidence in it decreased.)

  Whereas with the LSTM, the training loss drop nicely, but the validation one increased as I continue training. Again, this shows some over fitting in the LSTM network.

### USING CONVOLUTION 1D:

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    # words will be group into the size of the filter (here 5)
    # number of filter is 128, each for 5 words at a time
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# As the size of the input was 120 words, and a filter that is 5 words long will 
# shave off 2 words from the front and back, leaving us with 116. The 128 filters 
# that we specified will show up here as part of the convolutional layer.
model.summary()

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

NUM_EPOCHS = 10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

In [0]:
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

If we train with the convolutions now, we will see that our accuracy does even better than before with close to about 100% on training and around 80% on validation. But as before, our loss increases in the validation set, indicating potential overfilling. 

### USING BIDIRECTIONAL WITH GRU Unit

In [0]:
model = tf.keras.Sequential([
                          tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
                          tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
                          tf.keras.layers.Dense(6, activation = "relu"),
                          tf.keras.layers.Dense(1, activation = "sigmoid")
])

model.summary()

In [0]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

NUM_EPOCHS=10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

In [0]:
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

If I try a GRU layer instead, with a GRU being a different type of RNN, and I make it bidirectional, my network will have a 169,997 parameters. My training time will fall to 20 seconds per epoch, and my accuracy is again very good on training, and not too bad on validation but again, showing some overfitting.

### NOTE:

- For each technique, check on the time, the differnece between train and validation results. Overfitting?

- Remember that with text, you'll probably get a bit more overfitting than you would have done with images. Not least because you'll almost always have out of vocabulary words in the validation data set. That is words in the validation dataset that weren't present in the training, naturally leading to overfitting. These words can't be classified and, of course, you're going to have these overfitting issues, but see what you can do to avoid them.

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)